<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor


from scipy.stats import boxcox
from scipy.stats import yeojohnson, uniform
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, truncnorm, randint
from pprint import pprint
import random
from catboost import CatBoostRegressor
from itertools import groupby

import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import recall_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.base import clone
import datetime
from datetime import datetime, timedelta
import math

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from pprint import pprint

from bs4 import BeautifulSoup 
import requests

In [ ]:
def fill_missed(lst,j,year,page,id_anchor):
    #эта функция принимает на вход список, содержащий кортежи sell_id и признак - lst
    #также место sell_id в кортеже - j, также год - year
    #также - страницу - page;
    #также список sell_id, найденных на странице - id_anchor
    #сравниваются sell_id в списке lst и sell_id, найденные на странице
    #если в lst не все sell_id, значит, на странице не найден
    #признак для данного sell_id => для данного sell_id значение признака заполняется текстом 'Nan'

    
    lst1 = [x[j] for x in lst]
    missed = list(set(id_anchor)-set(lst1))
    for i in missed:
        if j:
            lst.insert(id_anchor.index(i),('Nan',i))
        else:
            lst.insert(id_anchor.index(i),(i,'Nan'))
    if len(lst) > len(id_anchor):
        print('year = {}, page = {}'.format(year,page))
        raise Exception(lst)

In [ ]:
def data_frame_form(df_result, list_to_turn,turn_to_int, id_order, name):
    #данная функция на вход принимает df_result - DataFrame, который будем дополнять новыми столбцами
    #столбцы (списки) передаются в list_to_turn, каждый список содержит кортежи с sell_id и значением признака
    #при этом в turn_to_int передается флаг - нужно ли перевести столбец из object в int (если 1, то нужно, 0 - не нужно)
    #в id_order передается место sell_id в кортеже (0 или 1)
    #в name передаются имена для признаков в итоговом dataframe (имена столбцов)
    

    df_new = pd.DataFrame(list_to_turn)
    df_r = df_result.copy()
    
    if id_order:
        df_new.rename(columns={0:name, 1:'sell_id'}, inplace=True)
    else:
        df_new.rename(columns={0:'sell_id', 1:name}, inplace=True)
    df_new.drop_duplicates(subset=['sell_id'],inplace = True)
    df_new['sell_id'] = df_new['sell_id'].astype(int)
    if turn_to_int:
        df_new[name] = df_new[name].astype(int)
    return df_r.merge(df_new, how='left', on='sell_id')

In [ ]:
def page_count(year):
    #в данной функции мы для каждого года в списке лет - year - получаем количество страниц на auto.ru
    #результат выводим в словарь, у которого ключи - годы, значения - количество страниц
    
    res = {}
    for i in year:
        
        url = 'https://auto.ru/cars/'+str(i)+'-year/all/?page=1&sort=fresh_relevance_1-desc&output_type=list'
        response = requests.get(url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        res[i] = int(re.findall(r'class="Button__text">([0-9]+)</span></span></a></span><div class="ListingPagination-module__sequenceControls', str(page))[0])
        
    return res

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 15
DIR_TRAIN  = '../input/den-shev/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'2020_12_25_0_20_out.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
test.car_url

Данные для обучения необходимо добыть самостоятельно. Просмотр параметр car_url в предоставленном test (см. выше) показывает, что данные взяты с auto.ru => оттуда же буду и данные для обучения брать.
Чтобы данных получилось побольше, буду брать данные за каждый год, начиная с 1980 по 2020.
Данные по ссылке вида - 'https://auto.ru/cars/1980-year/all/?page=1&sort=fresh_relevance_1-desc&output_type=list'

Вообще данные с auto.ru я уже выбрал в своем личном ноубуке на локальной машине, выгрузил в файл "2020_12_25_0_20_out.csv" и использую именно данные из файла, т.к. даже у меня на машине данные выгружаются и обрабатываются где-то 3 часа. Здесь - онлайн - будет раза в 2-2,5 дольше (ставил эксперимент).

Ниже публикую код, как собирал данные с auto.ru. При желании можно его запустить, но будет часов 5-6 отрабатывать.


In [ ]:
#в этом коде собираются данные для train



description = []
mileage = []
model_name = []
sell_id = []
vendor = []
own_num = []
pts_r = []
gt_r = []
st_r = []
custom_s = []
pg = {}
model_date = []
number_of_doors = []
vehicle_transmission = []
price = []
fuel_type = []
state = []

print('started at {}'.format(datetime.now()))

res = page_count(range(1980,1981))

for key,value in res.items():
    for page_c in range(1,value+1):
    
        url = 'https://auto.ru/cars/'+str(key)+'-year/all/?page='+str(page_c)+'&sort=fresh_relevance_1-desc&output_type=list'
        response = requests.get(url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        
        #обнаружил, что сразу много данных нужных можно удобно собрать в список pg1, см. ниже 
        
        pg1 = re.findall(r'meta content=\"(.*?)\" itemprop=\"(.*?)\"', str(page))
        
        #список лучше преобразовать в словарь pg1d, где ключи - название параметров, значения - значения параметро
        
        pg1d = {x[1]: [y[0] for y in pg1 if y[1] == x[1] ] for x in pg1}
        if key == 1980 and page_c == 1:
            pg = pg1d
        else:
            
            #здесь в итоговый словарь pg собираются данные с каждой страницы, т.е. в итоге в словаре для каждого ключа будет
            #список значений параметра, собранный со всех запрошенных страниц
            for key_1 in pg1d.keys():
                pg[key_1] = pg[key_1] + pg1d[key_1]
    
        #многие параметры удобно собираются сразу в словарь pg, но достаточное количество нужно отдельно выбирать со страницы
        #что как раз делаю ниже
        
        sell_id_t = re.findall(r',\"id\":\"([0-9]+)',str(page))
        if len(list(set(sell_id) & set(sell_id_t))) > 0:
            print('intersection of sell_id and sell_id_t')
            print(list(set(sell_id) & set(sell_id_t)))
        
        sell_id+=sell_id_t

        description_t = re.findall(r'delivery_info.*?\"description\":\"(.*?)\",\"discount_options.*?\"id\":\"([0-9]+)',str(page))
        fill_missed(description_t,1,key,page_c,sell_id_t)
        description+=description_t

        mileage_t = re.findall(r',\"id\":\"([0-9]+).*?\"mileage\":(.*?),\"state',str(page))
        fill_missed(mileage_t,0,key,page_c,sell_id_t)
        mileage+=mileage_t

        model_name_t = re.findall(r',\"id\":\"([0-9]+).*?\"model_info.*?name\":\"(.*?),\"ru_name',str(page))
        fill_missed(model_name_t,0,key,page_c,sell_id_t)
        model_name+=model_name_t

        vendor_t= re.findall(r',\"id\":\"([0-9]+).*?\"vendor\":\"(.*?)\"',str(page))
        fill_missed(vendor_t,0,key,page_c,sell_id_t)
        vendor+=vendor_t

        owners_number = re.findall(r'owners_number\":([0-9]+).*?\"id\":\"([0-9]+)',str(page))
        own_num_t = [('3 и более',x[1]) if int(x[0]) > 2 else x for x in owners_number]
        fill_missed(own_num_t,1,key,page_c,sell_id_t)
        own_num+=own_num_t

        pts= re.findall(r'pts\":\"(.*?)\".*?,\"id\":\"([0-9]+)"',str(page))
        pts_r_t = [('Оригинал',x[1]) if x[0] == 'ORIGINAL' else ('Дубликат',x[1]) for x in pts]
        fill_missed(pts_r_t,1,key,page_c,sell_id_t)
        pts_r+=pts_r_t

        gear_type= re.findall(r',\"id\":\"([0-9]+).*?\"gear_type\":\"(.*?)\"',str(page))
        gt_r_t = [(x[0],'передний') if 'forward' in x[1].lower() else (x[0],'задний') if 'rear' in x[1].lower() else (x[0],'полный') for x in gear_type]
        fill_missed(gt_r_t,0,key,page_c,sell_id_t)
        gt_r+=gt_r_t

        st_wheel = re.findall(r',\"id\":\"([0-9]+).*?\"steering_wheel\":\"(.*?)\"',str(page))
        st_r_t = [(x[0],'Левый') if 'left' in x[1].lower() else (x[0],'Правый') for x in st_wheel]
        fill_missed(st_r_t,0,key,page_c,sell_id_t)
        st_r+=st_r_t

        custom= re.findall(r'custom_cleared\":(.*?)\".*?,\"id\":\"([0-9]+)"',str(page))
        custom_s_t = [(1,x[1]) if 'true' in x[0].lower() else (0,x[1]) for x in custom]
        fill_missed(custom_s_t,1,key,page_c,sell_id_t)
        custom_s+=custom_s_t
        
        model_date_t = re.findall(r',\"id\":\"([0-9]+).*?\"super_gen.*?,\"year_from\":([0-9]+),',str(page))
        fill_missed(model_date_t,0,key,page_c,sell_id_t)
        model_date+=model_date_t

        nod_t = re.findall(r',\"id\":\"([0-9]+).*?\"vehicle_info.*?,\"doors_count\":([0-9]),',str(page))
        fill_missed(nod_t,0,key,page_c,sell_id_t)
        number_of_doors+=nod_t
        
        trans_t = re.findall(r',\"id\":\"([0-9]+).*?\"vehicle_info.*?,\"transmission\":\"(.*?)\",',str(page))
        fill_missed(trans_t,0,key,page_c,sell_id_t)
        vehicle_transmission+=trans_t
        
        price_t = re.findall(r',\"id\":\"([0-9]+).*?\"price\":([0-9]+),',str(page))
        fill_missed(price_t,0,key,page_c,sell_id_t)
        price+=price_t
        
        fuel_type_t = re.findall(r',\"id\":\"([0-9]+).*?\"engine_type\":\"(.*?)\",',str(page))
        fill_missed(fuel_type_t,0,key,page_c,sell_id_t)
        fuel_type+=fuel_type_t
        
        state_t = re.findall(r',\"id\":\"([0-9]+).*?\"state_not_beaten\":(.*?),',str(page))
        fill_missed(state_t,0,key,page_c,sell_id_t)
        state+=state_t

        print('page {}, year {}, at {}'.format(page_c,key,datetime.now()))
    
print('finished at {}'.format(datetime.now()))

В дальнейшем, когда я на всех собранных данных проверял, как данные собрались в общий словарь pg. Обнаружил:
<br>1) в поле fuelType собралось вдвое больше значений, чем в остальные;
<br>2) в поле name - втрое больше значений, чем в остальные;
<br>3) в поле price - количество значений отличается от остальных;
<br>4) в поле modelDate - количество значений отличается от остальных;
<br>5) в поле numberOfDoors - количество значений отличается от остальных;
<br>6) в поле vehicleTransmission - количество значений отличается от остальных;

1,2,3 - видно даже из примера ниже.
4,5,6 - проявляется, когда больше данных собираешь
При этом поля image, availability - по моему мнению, никакой полезной информации не дают => я их удалил.
Также я удалил из словаря pg 1,2,3,4,5,6, чтобы поискать их уже отдельно на странице и, если не найду, вместо них поставить 'Nan'.

Поле url содержит в себе sell_id, выделю его.


In [ ]:
for i in pg.keys():
    print('key is {}, len is {}'.format(i,len(pg[i])))

In [ ]:
#как писал выше, удаляю из словаря все выбранные для удаления параметры
dlist = ['fuelType','name','modelDate','numberOfDoors','vehicleTransmission','price','image','availability']
for i in dlist:
    pg.pop(i, None)


In [ ]:
#делаем из словаря dataframe, который будем потом дополнять данными из списков.
df_result = pd.DataFrame(pg)

In [ ]:
df_result

In [ ]:
#переименуем столбец "url" в "sell_id"
df_result.rename(columns={'url':'sell_id'}, inplace=True)
#преобразуем url в id
df_result['sell_id'] = df_result.sell_id.str.extract(r'/([0-9]+)-').astype(int)

In [ ]:
df_result

In [ ]:
#sell_id - уникальный идентификатор строк в получившемся dataframe (в test - тоже) =>
#удаляем из dataframe строки с повторяющимися sell_id

df_result.drop_duplicates(subset=['sell_id'],inplace = True)
df_result.sell_id.value_counts()

In [ ]:
#определим список параметров, в кортежах которых sell_id на 0 позиции
list_0_0 = [model_name,vendor,gt_r,st_r,vehicle_transmission,mileage,number_of_doors,price,model_date,fuel_type,state]
#задаем имена для столбцов, такие же, как у соответствующих столбцов у test 
list_0_0_names = ['model_name','vendor','Привод','Руль','vehicleTransmission','mileage','numberOfDoors','price','modelDate','fuelType','Состояние']

In [ ]:
# в рамках цикла добавляем к df_result столбцы из list_0_0, используем функцию data_frame_form

for i in range(0,len(list_0_0)):
    df_result = data_frame_form(df_result,list_0_0[i],0,0,list_0_0_names[i])

df_result

In [ ]:
#определим список параметров, в кортежах которых sell_id на 1 позиции
list_0_1 = [description,pts_r,custom_s,own_num]

#задаем имена для столбцов, такие же, как у соответствующих столбцов у test 
list_0_1_names = ['description','ПТС','Таможня','Владельцы']

# в рамках цикла добавляем к df_result столбцы из list_0_1, используем функцию data_frame_form

for i in range(0,len(list_0_1)):
    df_result = data_frame_form(df_result,list_0_1[i],0,1,list_0_1_names[i])

df_result

In [ ]:
df_result.info()

In [ ]:
#избавимся от пропусков в данных, делаю это, а не заполняю пропуски, потому что даже после всех чисток данных осталось > 68000 строк, 
#данных достаточно, а трудоемкость снизилась и данные настоящие, а не корректированные

df_result.dropna(inplace=True)


In [ ]:
# у нас также в каждом столбце есть текстовые значения "Nan", уберем строчки с такими значениями, 
# чтобы ни в одном столбце таких значений не осталось

for i in df_result.columns:
    df_result = df_result[df_result[i] != 'Nan']

In [ ]:
df_result

In [ ]:
#изменю у параметров 'productionDate','mileage','numberOfDoors','price','modelDate' на Int

df_result[['productionDate','mileage','numberOfDoors','price','modelDate']] = df_result[['productionDate','mileage','numberOfDoors','price','modelDate']].astype(int)

df_result.info()

все данные готовы, теперь их сравним попарно столбцы в test и df_result

Начнем с bodyType

In [ ]:
df_result.bodyType.value_counts()

In [ ]:
test.bodyType.value_counts()

данные готовы в df_result, ничего дополнительно делать не надо

смотрим brand                 

In [ ]:
df_result.brand.value_counts()

In [ ]:
test.brand.value_counts()

данные готовы в df_result, ничего дополнительно делать не надо

смотрим color

In [ ]:
df_result.color.value_counts()

In [ ]:
test.color.value_counts()

данные готовы в df_result, ничего дополнительно делать не надо

смотрим productionDate



In [ ]:
df_result.productionDate.value_counts()

In [ ]:
test.productionDate.value_counts()

данные готовы в df_result, ничего дополнительно делать не надо

смотрим vehicleConfiguration

In [ ]:
df_result.vehicleConfiguration.value_counts()

In [ ]:
test.vehicleConfiguration.value_counts()

данные готовы в df_result, ничего дополнительно делать не надо

смотрим priceCurrency

In [ ]:
df_result.priceCurrency.value_counts()

In [ ]:
test.priceCurrency.value_counts()

И в df_result (и в train, который я импортировал в csv), и в test в этом признаке только 1 значение => можно признак убрать, полезной информации не добавляет. Делаю ниже.

In [ ]:
test.drop(columns = ['priceCurrency'],inplace=True)
df_result.drop(columns = ['priceCurrency'],inplace=True)
test.info()

In [ ]:
df_result.info()

смотрим engineDisplacement

In [ ]:
df_result.engineDisplacement.value_counts()

In [ ]:
test.engineDisplacement.value_counts()

Данные похожи в обоих датасетах. Дополнительно можно убрать 'LTR' и сделать данные числовыми (float).

Сделаем в test и сразу в train (в моем csv я этого не сделал).

In [ ]:
train.engineDisplacement = train.engineDisplacement.str.replace('LTR','',regex=True).str.replace(' ','0',regex=True).astype(float)
test.engineDisplacement = test.engineDisplacement.str.replace('LTR','',regex=True).str.replace(' ','0',regex=True).astype(float)
test.engineDisplacement

смотрим enginePower

In [ ]:
test.enginePower.value_counts()

In [ ]:
df_result.enginePower.value_counts()

Данные похожи в обоих датасетах. Дополнительно можно убрать 'N12' и сделать данные числовыми (int).

Изначально, в данных, которые я собрал с auto.ru, я еще обнаружил, что в столбце enginePower было значение 'undefined N12'. Я это значение убрал следующим образом: df_result = df_result[df_result.enginePower != 'undefined N12']

Сделаем в test и сразу в train (в моем csv я этого не сделал).

In [ ]:
train.enginePower = train.enginePower.str.replace('N12','',regex=True).str.replace(' ','',regex=True).astype(int)
test.enginePower = test.enginePower.str.replace('N12','',regex=True).str.replace(' ','',regex=True).astype(int)
test.enginePower

смотрим model_name

In [ ]:
df_result.model_name.value_counts()

In [ ]:
test.model_name.value_counts()

как видно, значения в столбцах похожи. Только в df_result значения заканчиваются на ". Я это значение убрал следующим образом: df_result.model_name = df_result.model_name.str[:-1]. В csv, который загрузил, это уже сделал

смотрим vendor


In [ ]:
df_result.vendor.value_counts()

In [ ]:
test.vendor.value_counts()

Значения похожие, т.к. значений немного имеет смысл сделать one hot encoding. Но сделаю это уже после слияния датасетов.

смотрим Привод

In [ ]:
df_result.Привод.value_counts()

In [ ]:
test.Привод.value_counts()

Аналогично vendor - значений немного имеет смысл сделать one hot encoding. Но сделаю это уже после слияния датасетов.

смотрим Руль

In [ ]:
df_result.Руль.value_counts()

In [ ]:
test.Руль.value_counts()

Когда солью датасеты. "Левый" = 1, "Правый" = 0

смотрим Состояние


In [ ]:
df_result.Состояние.value_counts()

In [ ]:
test.Состояние.value_counts()

Состояния и в df_result(в train - также), и в test - единственное значение => удаляю столбец (в загруженном train уже удален). Пользы не несет.

In [ ]:
test.drop(columns = ['Состояние'],inplace=True)
df_result.drop(columns = ['Состояние'],inplace=True)
test.info()

смотрим vehicleTransmission

In [ ]:
df_result.vehicleTransmission.value_counts()

In [ ]:
test.vehicleTransmission.value_counts()

In [ ]:
#в train ошибся и вместо "роботизированная" заполнил соответствующие строчки значением 'роботизировання', исправляю ниже
train.vehicleTransmission[train.vehicleTransmission=='роботизировання'] = 'роботизированная'
train.vehicleTransmission.value_counts()

в df_result значения в латинице, нужно перевести, чтобы было аналогично тому, как в test (в загруженном train уже сделал)ю
Сделал так: 

lvt = [('механическая' if x[0] == 'M' else ('автоматическая' if x[0] == 'A' else ('вариатор' if x[0] == 'V' else 'роботизировання'))) for x in  list(df_result.vehicleTransmission.values)]
df_result.vehicleTransmission = lvt

после слияния датасетов сделаю one hot encoding.

смотрим mileage

In [ ]:
df_result.mileage.value_counts()

In [ ]:
test.mileage.value_counts()

столбцы можно совмещать. 
При выгрузке с сайта, у меня в столбце попадался "мусор" типа "0,"images_count":14},"sub_category":"cars"...", решил проблему следующим образом: df_result = df_result[df_result.mileage.str.match('^[0-9]+$')]

в загруженном train уже сделано.

Смотрим numberOfDoors

In [ ]:
df_result.numberOfDoors.value_counts()

In [ ]:
test.numberOfDoors.value_counts()

после слияния датасетов имеет смысл удалить строчку, в которой количество дверей  = 0

one hot encoding делать не буду, оставлю, как есть.

смотрим modelDate

In [ ]:
df_result.modelDate.value_counts()

In [ ]:
test.modelDate.value_counts()

При выгрузке с сайта, у меня в столбце попадался "мусор", решил проблему следующим образом: df_result = df_result[df_result.modelDate.str.match('^[0-9]+$')]

в загруженном train уже сделано.

Смотрим fuelType

In [ ]:
df_result.fuelType.value_counts()

In [ ]:
test.fuelType.value_counts()

в df_result значения в латинице, нужно перевести, чтобы было аналогично тому, как в test (в загруженном train уже сделал). Сделал так:

<br>ft = [('бензин' if x[0] == 'G' else ('дизель' if x[0] == 'D' else ('гибрид' if x[0] == 'H' else ('электро' if x[0] == 'E' else 'газ')))) for x in list(df_result.fuelType.values)]

<br>df_result.fuelType = ft

после слияния датасетов сделаю one hot encoding.

смотрим description

In [ ]:
df_result.description

In [ ]:
test.description

очень много текста, совсем неочевидно, как из этого извлечь пользу => удаляю столбцы в test и df_result. В train уже сделано.

In [ ]:
test.drop(columns = ['description'],inplace=True)
df_result.drop(columns = ['description'],inplace=True)
test.info()

Смотрим ПТС

In [ ]:
df_result.ПТС.value_counts()

In [ ]:
test.ПТС.value_counts()

Когда солью датасеты. "Оригинал" = 1, "Дубликат" = 0

смотрим Таможня

In [ ]:
df_result.Таможня.value_counts()

In [ ]:
test.Таможня.value_counts()

Таможня и в df_result(в train - также), и в test - единственное значение => удаляю столбец (в загруженном train уже удален). Пользы не несет.

In [ ]:
test.drop(columns = ['Таможня'],inplace=True)
df_result.drop(columns = ['Таможня'],inplace=True)
test.info()

Смотрим Владельцы

In [ ]:
df_result.Владельцы.value_counts()

In [ ]:
test.Владельцы.value_counts()

нужно, чтобы у train были такие же значения, как и у test

In [ ]:
for i in range(0,len(train.Владельцы.value_counts().index)):
   
    train.Владельцы[train.Владельцы == train.Владельцы.value_counts().index[i]] = test.Владельцы.value_counts().index[i]

In [ ]:
train.Владельцы.value_counts()

после слияния сделаю one hot encoding.

все совпадающие поля у train и test проверили и обработали.
Выясним, какие у test остались столбцы, которых нет у train

In [ ]:
list(set(list(test.columns)) - set(list(train.columns)))

первое, что необходимо сделать - переименовать столбец "id" в "sell_id" у train. Делаю

In [ ]:
train.rename(columns={'id':'sell_id'}, inplace=True)

In [ ]:
list(set(list(test.columns)) - set(list(train.columns)))

рассмотрим столбец equipment_dict у test

In [ ]:
test.equipment_dict

содержит словари, в которых много данных, причем, ключи неоднородные в словарях => будет сложно пользу извлечь из такого параметра => удаляю

In [ ]:
test.drop(columns = ['equipment_dict'],inplace=True)

столбец image надо удалять, не глядя). Удаляю.

In [ ]:
test.drop(columns = ['image'],inplace=True)

рассмотрим столбец super_gen

In [ ]:
test.super_gen

словарь со множеством ключей, которые не одинаковые в разных строках => пользу будет очень тяжело извлечь. Удаляю

In [ ]:
test.drop(columns = ['super_gen'],inplace=True)

рассмотрим столбец model_info

In [ ]:
test.model_info

словарь, который содержит данные из столбцов model_name, brand => словарь + избыточная информация => удаляю.

In [ ]:
test.drop(columns = ['model_info'],inplace=True)

рассмотрим столбец name

In [ ]:
test.name

в столбце собраны данные из других столбцов engineDisplacement, enginePower => избыточность, удаляю.

In [ ]:
test.drop(columns = ['name'],inplace=True)

рассмотрим столбец Владение

In [ ]:
test.Владение.value_counts(normalize=True,dropna=False)

в столбце 65% значений пропущено => удаляю столбец

In [ ]:
test.drop(columns=['Владение'],inplace=True)

столбец car_url - считаю, что от ссылки пользы нет, она содержит sell_id, который у нас выделен в отдельный столбец => удаляю.

In [ ]:
test.drop(columns=['car_url'],inplace=True)

рассмотрим столбец complectation_dict

In [ ]:
test.complectation_dict.value_counts(normalize=True,dropna=False)

содержит страшные словари) + 81% - пропуски, удаляю столбец вообще без сожалений).

In [ ]:
test.drop(columns = ['complectation_dict'],inplace=True)

столбец parsing_unixtime - юниксовое время, когда test распарсили - не понимаю, какая польза от этого может быть => удаляю

In [ ]:
test.drop(columns = ['parsing_unixtime'],inplace = True)

In [ ]:
list(set(list(test.columns)) - set(list(train.columns)))

мы готовы к слиянию датасетов

train['Train'] = 1 # помечаем где у нас трейн
test['Train'] = 0 # помечаем где у нас тест

df = pd.concat([df, test], axis=0).reset_index(drop=True)
df.sample()

In [ ]:
train['Train'] = 1 # помечаем где у нас трейн 
test['Train'] = 0 # помечаем где у нас тест

df = pd.concat([train, test], axis=0).reset_index(drop=True) 
df.sample()

проверим, что в итоговом датасете получилось

In [ ]:
df.info()
df.isnull().sum()

как видим, в итоговом датасете отсуствуют price - это понятно, т.к. в test не было столбца с price. Также отсутствует 1 значение в столбце "ПТС" => заполню самым распространенным значением

In [ ]:
df.ПТС[df.ПТС.isnull()] = 'Оригинал'

In [ ]:
df.info()

## Разделим данные на группы переменных:¶


По итогам рассмотрения столбцов, решил:

<br>1) vendor, Привод, vehicleTransmission, fuelType, Владельцы - применю One hot encoding

<br>2) Руль, ПТС - binary кодирование:

Руль -  "Левый" = 1, "Правый" = 0
ПТС - "Оригинал" = 1, "Дубликат" = 0

<br>3) productionDate, engineDisplacement, enginePower, mileage,numberOfDoors, modelDate - числовые столбцы;

<br> 4) bodyType, brand, color, vehicleConfiguration, model_name - категориальные переменные, которые закодирую через data[colum].astype('category').cat.codes

In [ ]:
one_hot_cols = ['vendor', 'Привод', 'vehicleTransmission', 'fuelType', 'Владельцы']
cat_cols = ['bodyType', 'brand', 'color', 'model_name','vehicleConfiguration'] # категориальные переменные
num_cols = ['modelDate', 'productionDate', 'mileage', 'numberOfDoors',
            'enginePower','engineDisplacement'] # числовые переменные

## Числовые признаки

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(df[i])
    plt.title(i)
    plt.show()

In [ ]:
# Посмотрим корреляцию числовых признаков
correlation = df[num_cols].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
df.enginePower

In [ ]:
df.engineDisplacement

Из графика видно:
<br>+0.95 между modelDate и productionDate. Для анализа оставим productionDate
<br>+0.81 между enginePower и engineDisplacement. Оставлю engineDisplacement (меньше разлет значений)

In [ ]:
df1 = df.drop(columns = ['modelDate','enginePower'])
df1.info()

In [ ]:
#делаем One hot кодирование для выбранных столбцов
for i in one_hot_cols:
    dummies = pd.get_dummies(df1[i],prefix=i)
    for j in dummies.columns:
        df1[j] = dummies[j]
    df1.drop(columns = i,inplace = True)

In [ ]:
df1.info()

кодируем категориальные признаки

In [ ]:
for colum in cat_cols:
    df1[colum] = df1[colum].astype('category').cat.codes

In [ ]:
df1.info()


кодируем бинарные Руль и ПТС

Руль - "Левый" = 1, "Правый" = 0 ПТС - "Оригинал" = 1, "Дубликат" = 0

In [ ]:
df1.Руль[df1.Руль == 'Левый'] = 1
df1.Руль[df1.Руль == 'Правый'] = 0
df1.Руль = df1.Руль.astype(int)
df1.ПТС[df1.ПТС == 'Оригинал'] = 1
df1.ПТС[df1.ПТС == 'Дубликат'] = 0
df1.ПТС = df1.ПТС.astype(int)

In [ ]:
df1.Руль.value_counts()

In [ ]:
df1.ПТС.value_counts()

In [ ]:
df1.info()

In [ ]:
#сохраняем значения целевого параметра
y = df1['price'][df1['Train'] == 1]

In [ ]:
df1.info()

In [ ]:
X = df1[df1.Train == 1].drop(columns = ['Train','sell_id','price'])
X_sub = df1[df1.Train == 0].drop(columns = ['Train','sell_id','price'])

In [ ]:
X_sub

## Train Split

In [ ]:
VAL_SIZE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.save_model('catboost_single_model_baseline.model')

In [ ]:
predict = model.predict(X_test)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 30.05%

# Random Forest

In [ ]:
rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train)
test_predict = rf.predict(X_test)
print(f"Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

# Bagging

In [ ]:
tree = DecisionTreeRegressor(max_features=int(X_train.shape[1]/3), max_depth=50)
tree.fit(X_train, y_train)
test_predict = tree.predict(X_test)
print(f"DecisionTreeRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# GradientBoosting

In [ ]:
gb = GradientBoostingRegressor(max_depth=7,n_estimators=1000,learning_rate=0.1,random_state=42)
gb.fit(X_train, y_train)
test_predict = gb.predict(X_test)
print(f"GradientBoostingRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

# Submission

In [ ]:
predict_submission = gb.predict(X_sub)
predict_submission

In [ ]:
X_sub

лучше всего себя показала GradientBoosting, буду сдаваться с этой моделью.

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)